<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

## Model Submission Guide: Titanic Survival Competition
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data using Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/titanic_competition_data-repository:v1') 


Data downloaded successfully.


In [3]:
# Separate data into X_train, y_train, and X_test
import pandas as pd
training_data=pd.read_csv("titanic_competition_data/training_data.csv")
y_train= pd.get_dummies(training_data['survived'])
X_train = training_data.drop(['survived'], axis=1)

X_test=pd.read_csv("titanic_competition_data/test_data.csv")

X_train.head()

,pclass,sex,age,fare,embarked
0,3,male,28.0,7.2500,S
1,3,female,26.0,16.1000,S
2,3,female,47.0,7.0000,S
3,2,male,57.0,12.3500,Q
4,3,female,37.0,9.5875,S


##2.   Preprocess data using Sklearn Column Transformer/ Write and Save Preprocessor function


In [4]:
# In this case we use Sklearn's Column transformer in our preprocessor function

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Preprocess data using sklearn's Column Transformer approach

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #'imputer' names the step
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']

# Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [5]:
# Here is where we actually write the preprocessor function:

# Write function to transform data with preprocessor 
# In this case we use sklearn's Column transformer in our preprocessor function

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [6]:
# check shape of X data after preprocessing it using our new function
preprocessor(X_train).shape

(1047, 10)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(64, input_dim=10, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(2, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), pd.get_dummies(y_train), 
               batch_size = 20, 
               epochs = 3, validation_split=0.25) 

Epoch 1/3
40/40 [==============================] - 2s 21ms/step - loss: 0.6727 - accuracy: 0.6038 - val_loss: 0.6402 - val_accuracy: 0.7061
Epoch 2/3
40/40 [==============================] - 0s 9ms/step - loss: 0.6330 - accuracy: 0.6879 - val_loss: 0.6073 - val_accuracy: 0.7176
Epoch 3/3
40/40 [==============================] - 0s 7ms/step - loss: 0.6086 - accuracy: 0.6955 - val_loss: 0.5823 - val_accuracy: 0.7405


#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

#This is the unique rest api that powers this Titanic Survival Playground -- make sure to update the apiurl for new competition deployments
apiurl="https://n0l8kcy3wh.execute-api.us-east-1.amazonaws.com/prod/m"

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1327


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,relu_act,softmax_act,loss,optimizer,model_config,memory_size,username,version
0,79.77%,78.44%,79.24%,77.97%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",nan,AIModelShare,1
1,77.10%,73.88%,79.17%,72.96%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,lbfgs,"{'C': 0.01, 'class_weight': No...",nan,AIModelShare,2
2,55.73%,50.10%,51.18%,50.97%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,lbfgs,"{'C': 0.01, 'class_weight': No...",nan,AIModelShare,4
3,53.05%,49.97%,50.07%,50.07%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",nan,AIModelShare,3
4,53.82%,48.66%,49.28%,49.39%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,AIModelShare,5
5,51.53%,48.75%,48.77%,48.80%,sklearn,False,False,GradientBoostingClassifier,nan,nan,nan,nan,nan,nan,nan,"{'ccp_alpha': 0.0, 'criterion'...",nan,AIModelShare,6
6,57.25%,42.11%,46.50%,48.78%,keras,False,True,Sequential,4.000000,9154.000000,4.000000,3.000000,1.000000,str,SGD,"{'name': 'sequential', 'layers...",1322832.000000,AIModelShare,7


## 5. Repeat submission process to improve place on leaderboard


In [14]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model_2 = Sequential()
model_2.add(Dense(128, input_dim=10, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))

model_2.add(Dense(2, activation='softmax')) 
                                            
# Compile model
model_2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model_2.fit(preprocessor(X_train), pd.get_dummies(y_train), 
               batch_size = 20, 
               epochs = 5, validation_split=0.25) 

Epoch 1/5
40/40 [==============================] - 1s 7ms/step - loss: 0.6912 - accuracy: 0.5427 - val_loss: 0.6698 - val_accuracy: 0.6107
Epoch 2/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6709 - accuracy: 0.6000 - val_loss: 0.6491 - val_accuracy: 0.6374
Epoch 3/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6604 - accuracy: 0.6153 - val_loss: 0.6328 - val_accuracy: 0.6603
Epoch 4/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6432 - accuracy: 0.6127 - val_loss: 0.6140 - val_accuracy: 0.6718
Epoch 5/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6292 - accuracy: 0.6280 - val_loss: 0.5964 - val_accuracy: 0.6985


In [15]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [16]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model_2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 8

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1327


In [17]:
# Compare two or more models
data=mycompetition.compare_models([7,8], verbose=1)
mycompetition.stylize_compare(data)

,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params
0,Dense,"[None, 64]",704.000000,Dense,"[None, 128]",1408
1,Dense,"[None, 64]",4160.000000,Dropout,"[None, 128]",0
2,Dense,"[None, 64]",4160.000000,Dense,"[None, 64]",8256
3,Dense,"[None, 2]",130.000000,Dense,"[None, 64]",4160
4,None,None,nan,Dropout,"[None, 64]",0
5,None,None,nan,Dense,"[None, 64]",4160
6,None,None,nan,Dense,"[None, 2]",130


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [19]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt


#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(64, input_dim=10, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                    activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(2, activation='softmax')) 
    model.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=3, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=1, validation_data=(preprocessor(x_val), y_val))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.7746031681696574

Best val_accuracy So Far: 0.7746031681696574
Total elapsed time: 00h 00m 14s


In [21]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 1s 3ms/step - loss: 1.8803 - accuracy: 0.7077
Epoch 2/5
33/33 [==============================] - 0s 2ms/step - loss: 1.2627 - accuracy: 0.7660
Epoch 3/5
33/33 [==============================] - 0s 2ms/step - loss: 0.9501 - accuracy: 0.7717
Epoch 4/5
33/33 [==============================] - 0s 2ms/step - loss: 0.7757 - accuracy: 0.7880
Epoch 5/5
33/33 [==============================] - 0s 2ms/step - loss: 0.6817 - accuracy: 0.7803


In [22]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [23]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 9

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1327


In [24]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,dropout_layers,relu_act,softmax_act,loss,optimizer,model_config,memory_size,username,version
0,79.77%,78.44%,79.24%,77.97%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",nan,AIModelShare,1
1,77.10%,73.88%,79.17%,72.96%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,lbfgs,"{'C': 0.01, 'class_weight': No...",nan,AIModelShare,2
2,55.73%,50.10%,51.18%,50.97%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,lbfgs,"{'C': 0.01, 'class_weight': No...",nan,AIModelShare,4
3,61.07%,44.88%,58.82%,52.11%,keras,False,True,Sequential,7.000000,18114.000000,5.000000,2.000000,4.000000,1.000000,str,SGD,"{'name': 'sequential_1', 'laye...",1553904.000000,AIModelShare,8
4,53.05%,49.97%,50.07%,50.07%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",nan,AIModelShare,3
5,53.82%,48.66%,49.28%,49.39%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",nan,AIModelShare,5
6,53.44%,48.65%,49.12%,49.23%,keras,False,True,Sequential,4.000000,19874.000000,4.000000,nan,3.000000,1.000000,str,RMSprop,"{'name': 'sequential_1', 'laye...",2103472.000000,AIModelShare,9
7,51.53%,48.75%,48.77%,48.80%,sklearn,False,False,GradientBoostingClassifier,nan,nan,nan,nan,nan,nan,nan,nan,"{'ccp_alpha': 0.0, 'criterion'...",nan,AIModelShare,6
8,57.25%,42.11%,46.50%,48.78%,keras,False,True,Sequential,4.000000,9154.000000,4.000000,nan,3.000000,1.000000,str,SGD,"{'name': 'sequential', 'layers...",1322832.000000,AIModelShare,7


In [25]:
# Compare two or more models
data=mycompetition.compare_models([7,8,9], verbose=1)
mycompetition.stylize_compare(data)

,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params,Model_9_Layer,Model_9_Shape,Model_9_Params
0,Dense,"[None, 64]",704.000000,Dense,"[None, 128]",1408,Dense,"[None, 64]",704.000000
1,Dense,"[None, 64]",4160.000000,Dropout,"[None, 128]",0,Dense,"[None, 64]",4160.000000
2,Dense,"[None, 64]",4160.000000,Dense,"[None, 64]",8256,Dense,"[None, 224]",14560.000000
3,Dense,"[None, 2]",130.000000,Dense,"[None, 64]",4160,Dense,"[None, 2]",450.000000
4,None,None,nan,Dropout,"[None, 64]",0,None,None,nan
5,None,None,nan,Dense,"[None, 64]",4160,None,None,nan
6,None,None,nan,Dense,"[None, 2]",130,None,None,nan
